# Potential Player
## Objective
Identify users not playing any Nexon game, but have high likelihood of playing one of Nexon's game

## Methodology
Nexon has 3 games, each with its own player profile. We will try to identify players sharing similar profile, such as purchasing and gaming behavior.  
The assumption is that if, for example, most MapleStory players only spend about 30 hours playing a week, it could mean that the nature of the game or the gaming community is suitable for people who can devote 30 hours a week playing game. Thus, people who do not spend around the same time are likely not suitable for MapleStory and therefore have less likelihood of playing MapleStory.  
Here we will create user profile (features) and build binary classification model to see the likelihood of players for playing each Nexon game (MapltStory, Mabinogi, and Vindictus). Our target here are players who have high probability but had not play at Nexon game before.

## Steps
1. Label games with relevant genre to slim down number of available features
2. feature genaration such as purchase genre Y, play time for genre X, % time playing genre X, # of games played, ...etc.
3. model building

## Brainstorm/Notes
1. look into people who play low-res games (games with earlier published days) as they may have hardware limtations
2. look into people who play exclusively MMORPG